# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
#from tensorflow import keras
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Embedding
#from tensorflow.keras.layers import SimpleRNN
#from tensorflow.keras.datasets import imdb
#from tensorflow.keras import initializers
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

In [2]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [3]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
C:\Users\KonuTech\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


C:\Users\KonuTech\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [4]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [5]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99])

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [6]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [7]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 50)          1000000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 5)                 280       
_________________________________________________________________
dense (Dense)                (None, 1)                 6         
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [8]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [9]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 8s 9ms/step - loss: 0.6790 - accuracy: 0.5691 - val_loss: 0.5975 - val_accuracy: 0.6815
Epoch 2/10
782/782 [==============================] - 6s 7ms/step - loss: 0.5630 - accuracy: 0.7151 - val_loss: 0.5346 - val_accuracy: 0.7292
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.5007 - accuracy: 0.7586 - val_loss: 0.5109 - val_accuracy: 0.7422
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4477 - accuracy: 0.7921 - val_loss: 0.4819 - val_accuracy: 0.7654
Epoch 5/10
782/782 [==============================] - 6s 7ms/step - loss: 0.4215 - accuracy: 0.8074 - val_loss: 0.4673 - val_accuracy: 0.7737
Epoch 6/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3996 - accuracy: 0.8176 - val_loss: 0.4575 - val_accuracy: 0.7806
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.3883 - accuracy: 0.8248 - val_loss: 0.4543 - val_accuracy: 0.7825
Epoch 

In [10]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 1s 1ms/step - loss: 0.4479 - accuracy: 0.7908
Test score: 0.44786936044692993
Test accuracy: 0.7907999753952026


## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [11]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [12]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [13]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [14]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 12s 14ms/step - loss: 0.6670 - accuracy: 0.5771 - val_loss: 0.5403 - val_accuracy: 0.7459
Epoch 2/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4918 - accuracy: 0.7726 - val_loss: 0.4677 - val_accuracy: 0.7806
Epoch 3/10
782/782 [==============================] - 11s 14ms/step - loss: 0.4034 - accuracy: 0.8236 - val_loss: 0.4282 - val_accuracy: 0.8027
Epoch 4/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3603 - accuracy: 0.8467 - val_loss: 0.4044 - val_accuracy: 0.8178
Epoch 5/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3246 - accuracy: 0.8659 - val_loss: 0.4405 - val_accuracy: 0.7901
Epoch 6/10
782/782 [==============================] - 11s 14ms/step - loss: 0.3084 - accuracy: 0.8690 - val_loss: 0.3906 - val_accuracy: 0.8255
Epoch 7/10
782/782 [==============================] - 11s 14ms/step - loss: 0.2954 - accuracy: 0.8773 - val_loss: 0.3794 - val_accuracy:

In [15]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 11s 13ms/step - loss: 0.6702 - accuracy: 0.5773 - val_loss: 0.5463 - val_accuracy: 0.7218
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.5133 - accuracy: 0.7492 - val_loss: 0.5099 - val_accuracy: 0.7328
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4377 - accuracy: 0.7990 - val_loss: 0.4308 - val_accuracy: 0.7981
Epoch 4/10
782/782 [==============================] - 10s 12ms/step - loss: 0.3943 - accuracy: 0.8218 - val_loss: 0.4149 - val_accuracy: 0.8083
Epoch 5/10
782/782 [==============================] - 10s 12ms/step - loss: 0.3673 - accuracy: 0.8376 - val_loss: 0.4013 - val_accuracy: 0.8162
Epoch 6/10
782/782 [==============================] - 10s 12ms/step - loss: 0.3512 - accuracy: 0.8456 - val_loss: 0.4453 - val_accuracy: 0.7937
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3374 - accuracy: 0.8517 - val_loss: 0.3922 - val_accuracy:

In [19]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3052 - accuracy: 0.8705 - val_loss: 0.3682 - val_accuracy: 0.8364
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2986 - accuracy: 0.8764 - val_loss: 0.3680 - val_accuracy: 0.8368
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2934 - accuracy: 0.8769 - val_loss: 0.3676 - val_accuracy: 0.8371
Epoch 4/10
782/782 [==============================] - 10s 12ms/step - loss: 0.2891 - accuracy: 0.8772 - val_loss: 0.3612 - val_accuracy: 0.8396
Epoch 5/10
782/782 [==============================] - 10s 12ms/step - loss: 0.2854 - accuracy: 0.8801 - val_loss: 0.3622 - val_accuracy: 0.8407
Epoch 6/10
782/782 [==============================] - 10s 12ms/step - loss: 0.2820 - accuracy: 0.8823 - val_loss: 0.3643 - val_accuracy: 0.8402
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2786 - accuracy: 0.8839 - val_loss: 0.3608 - val_accuracy:

---
### Machine Learning Foundation (C) 2020 IBM Corporation